In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import os
from dotenv import load_dotenv
import unicodedata
from src.etl import load as lo #con jupyter
from src.eda import explore as exp  #con jupyter

In [2]:
load_dotenv()

True

In [3]:
#Llamamos a la variable de entorno
DB_NAME = os.getenv("DB_NAME")
DB_USER=os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST= os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [4]:
conn, cur = lo.crear_conexión()
#Comprobamos que la conexión está bien realizada:
cur.execute("SELECT version();")
cur.fetchone() #para que muestre el resultado de la query anterior
lo.cerrar_conexion(conn, cur)

# 1. Extracción de los datos de la BBDD

## Paises

In [5]:
query_extraccion = '''SELECT *
                                    FROM pais_destino'''        

In [9]:
lista_columnas = ['id_pais_destino','nombre_pais_destino', 'nombre_continente']
df_paises = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_paises

,id_pais_destino,nombre_pais_destino,nombre_continente
0,1,Albania,Europa
1,2,Alemania,Europa
2,3,Andorra,Europa
3,4,Angola,África
4,5,Antigua,América
...,...,...,...
179,180,Yemen,Asia
180,181,Zambia,África
181,182,Zanzibar,África
182,183,Zimbabue,África


## Viajes localizados según página web

In [14]:
query_extraccion = '''SELECT pd.*,
                                                        v.*
                                                    FROM viaje v
                                                    INNER JOIN combinacion_destino_viaje cdv ON v.id_viaje = cdv.id_viaje
                                                    INNER JOIN pais_destino pd ON cdv.id_pais_destino =pd.id_pais_destino'''     

In [15]:
lista_columnas = ['id_pais_destino', 'nombre_pais_destino', 'nombre_continente', 'id_viaje', 'url_viaje', 
                  'nombre_viaje','duracion_dias', 'duracion_noches', 'id_itinerario', 'viaje_activo']

In [16]:
df_viajes_localizacion_pag_web = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viajes_localizacion_pag_web

,id_pais_destino,nombre_pais_destino,nombre_continente,id_viaje,url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo
0,1,Albania,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1016.0,True
1,2,Alemania,Europa,2,https://viajeonline.es.tui.com/es/idea/1463025...,Berlín,4,3,144.0,True
2,2,Alemania,Europa,3,https://viajeonline.es.tui.com/es/idea/7897512...,"Fly & Drive Austria, Entre Lagos y Montañas",8,7,863.0,True
3,2,Alemania,Europa,4,https://viajeonline.es.tui.com/es/idea/8404511...,Fly & Drive Carlomagno a su aire,10,9,163.0,True
4,2,Alemania,Europa,5,https://viajeonline.es.tui.com/es/idea/7906930...,Fly & Drive Tirol al Completo,8,7,1131.0,True
...,...,...,...,...,...,...,...,...,...,...
1750,116,Montenegro,Europa,1437,https://viajeonline.es.tui.com/es/idea/2701570...,Secretos de los Balcanes,9,8,1143.0,True
1751,116,Montenegro,Europa,1438,https://viajeonline.es.tui.com/es/idea/2701052...,Grandeza de los Balcanes,15,14,1146.0,True
1752,154,Serbia,Europa,1438,https://viajeonline.es.tui.com/es/idea/2701052...,Grandeza de los Balcanes,15,14,1146.0,True
1753,112,Mexico,América,1442,https://viajeonline.es.tui.com/es/idea/2700326...,Ciudad de México y Holbox,9,7,1140.0,True


## Viajes según localización ciudades

In [17]:
query_extraccion = '''SELECT pd.*,
	v.*,
	c.id_ciudad,
	c.nombre_ciudad
FROM viaje v
INNER JOIN itinerario i ON v.id_itinerario = i.id_itinerario
INNER JOIN ciudad_itinerario ci ON i.id_itinerario = ci.id_itinerario
INNER JOIN ciudad c ON ci.id_ciudad = c.id_ciudad
INNER JOIN pais_destino pd ON c.id_pais_destino =pd.id_pais_destino'''


In [18]:
lista_columnas = ['id_pais_destino', 'nombre_pais_destino', 'nombre_continente', 'id_viaje', 'url_viaje', 
                  'nombre_viaje','duracion_dias', 'duracion_noches', 'id_itinerario', 'viaje_activo', 'id_ciudad', 'nombre_ciudad']

In [19]:
df_viajes_localizacion_ciudad = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viajes_localizacion_ciudad

,id_pais_destino,nombre_pais_destino,nombre_continente,id_viaje,url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo,id_ciudad,nombre_ciudad
0,116,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1016,True,9,Bar
1,116,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1016,True,8,Cetinje
2,116,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1016,True,7,Budva
3,116,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1016,True,6,Petrovac
4,1,Albania,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1016,True,5,Shkoder
...,...,...,...,...,...,...,...,...,...,...,...,...
7212,154,Serbia,Europa,1440,https://viajeonline.es.tui.com/es/idea/2700646...,Croacia Maravillosa,8,7,1145,True,2054,Trogir
7213,41,Croacia,Europa,1440,https://viajeonline.es.tui.com/es/idea/2700646...,Croacia Maravillosa,8,7,1145,True,419,Zadar
7214,154,Serbia,Europa,1440,https://viajeonline.es.tui.com/es/idea/2700646...,Croacia Maravillosa,8,7,1145,True,2053,Lagos De Plitvice
7215,41,Croacia,Europa,1440,https://viajeonline.es.tui.com/es/idea/2700646...,Croacia Maravillosa,8,7,1145,True,416,Zagreb


## Precio de los viajes

In [20]:
query_extraccion = '''SELECT v.*,
	precio_viaje,
	fecha_precio_viaje
FROM viaje v
INNER JOIN precio_viaje pv ON v.id_viaje = pv.id_viaje'''

In [21]:
lista_columnas = [ 'id_viaje', 'url_viaje', 'nombre_viaje','duracion_dias', 'duracion_noches', 'id_itinerario', 
                  'viaje_activo', 'precio_viaje', 'fecha_precio_viaje']

In [22]:
df_viaje_precios = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viaje_precios

,id_viaje,url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo,precio_viaje,fecha_precio_viaje
0,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1016.0,True,1755,2025-04-07
1,2,https://viajeonline.es.tui.com/es/idea/1463025...,Berlín,4,3,144.0,True,708,2025-04-07
2,3,https://viajeonline.es.tui.com/es/idea/7897512...,"Fly & Drive Austria, Entre Lagos y Montañas",8,7,863.0,True,827,2025-04-07
3,4,https://viajeonline.es.tui.com/es/idea/8404511...,Fly & Drive Carlomagno a su aire,10,9,163.0,True,857,2025-04-07
4,5,https://viajeonline.es.tui.com/es/idea/7906930...,Fly & Drive Tirol al Completo,8,7,1131.0,True,894,2025-04-07
...,...,...,...,...,...,...,...,...,...
47236,1366,https://es.tui.com/viaje/paisajes-y-tribus-de-...,Paisajes y Tribus de Vietnam y Camboya,17,14,452.0,True,3284,2025-05-19
47237,1367,https://es.tui.com/viaje/paisajes-y-tribus-de-...,Paisajes y Tribus de Vietnam y Phu Quoc,17,14,446.0,True,3310,2025-05-19
47238,1368,https://es.tui.com/viaje/paisajes-y-tribus-de-...,Paisajes y Tribus de Vietnam y Nha Trang,17,14,445.0,True,3450,2025-05-19
47239,1369,https://es.tui.com/viaje/zimbabue-salvaje,Zimbabue Salvaje,11,8,231.0,True,6539,2025-05-19


## Viajes con cambios de precios

In [36]:
query_extraccion = '''SELECT v.id_viaje,
	count(DISTINCT precio_viaje) as num_precios_ofertados
FROM viaje v
INNER JOIN precio_viaje pv ON v.id_viaje = pv.id_viaje
GROUP BY v.id_viaje
HAVING count(DISTINCT precio_viaje)>1;'''

In [37]:
lista_columnas = [ 'id_viaje', 'num_precios_ofertados']

In [38]:
df_viaje_con_precios_distintos = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viaje_con_precios_distintos

,id_viaje,num_precios_ofertados
0,11,2
1,12,2
2,15,2
3,16,2
4,17,2
5,19,2
6,20,2
7,21,2
8,23,2
9,24,2


## Turismo emisor

In [54]:
query_extraccion = '''SELECT pd.*,
	ccaa_origen,
	num_turistas,
	num_pernoctaciones,
	ano,
	mes
FROM turismo_emisor te 
INNER JOIN pais_destino pd ON te.id_pais_destino = pd.id_pais_destino
ORDER BY ano,mes ;'''

In [52]:
lista_columnas = [ 'id_pais_destino', 'nombre_pais_destino', 'nombre_continente', 'ccaa_origen','num_turistas', 'num_pernoctaciones', 'ano', 'mes']

In [55]:
df_turismo_emisor = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_turismo_emisor

,id_pais_destino,nombre_pais_destino,nombre_continente,ccaa_origen,num_turistas,num_pernoctaciones,ano,mes
0,8,Argentina,América,Andalucía,419,3616,2019,7
1,22,Bolivia,América,Andalucía,61,486,2019,7
2,25,Brasil,América,Andalucía,277,2214,2019,7
3,32,Canada,América,Andalucía,483,6159,2019,7
4,34,Chile,América,Andalucía,242,2068,2019,7
...,...,...,...,...,...,...,...,...
71092,155,Seychelles,África,Total Nacional,146,1105,2024,12
71093,160,Sudafrica,África,Total Nacional,986,9019,2024,12
71094,166,Tanzania,África,Total Nacional,590,5237,2024,12
71095,170,Tunez,África,Total Nacional,975,5012,2024,12


In [39]:
#conn.rollback()

In [60]:
df_turismo_emisor_agrupado = df_turismo_emisor.groupby('nombre_pais_destino')[['num_turistas', 'num_pernoctaciones']].sum()
df_turismo_emisor_agrupado

,num_turistas,num_pernoctaciones
nombre_pais_destino,,
Albania,180212,1115910
Alemania,11949860,68516775
Andorra,9323631,24769193
Angola,9669,79896
Antigua,220,1900
...,...,...
Venezuela,106829,803330
Vietnam,130169,1477676
Yemen,222,2128
